 # 実験2（数式の処理）
 - 前回の実験をもとに数式を構文解析し抽象構文木に変換する．
 - 抽象構文木を基に数式の値を計算する．

 ## 電卓の計算

 以下は前回と同じ（ライブラリの読み込みと10進整数を整数として切り出すパーサnumの定義）

In [0]:
from toyparsing import *

@parser_do
def num(run):
    return int(run(pat("[0-9]+")))

 数式は前回のCSVと同じくカツサンドパターン（数が「パン」で演算子(+,-,*,/)が「カツ」）である．
 よって数式を切り出すプログラム`expr0`は前回の`csv`と同様にできる（コンマが演算子に置き換わっただけ）．

In [0]:
@parser_do
def expr0(run):
    a = [run(num)]
    while True:
        r = run(pat("[+*/-]") & num, nullable=True) 
        if r is None: break
        a += r
    return a

In [0]:
expr0("1+2*3")

In [0]:
expr0("1+2*3-4/5")

 演算が左から順におこなわれることを明確にするために入れ子になったリストを用いてもよい．

In [0]:
@parser_do
def expr0(run):
    a = run(num)
    while True:
        r = run(pat("[+*/-]") & num, nullable=True) 
        if r is None: break
        [op, b] = r
        a = [op, a, b]
    return a

上記の例をもう一度実行してみるとかけ算よりたし算のほうが先にひとつにまとまっていることがわかる．
このように電卓では数学の習慣は無視して，単純に左から順に計算がおこなわれる．

 ## 抽象構文木
入れ子になったリストは**抽象構文木**（AST）を表現しているとみなすと視覚的にも分かりやすい．例えば`['*', ['+', 1, 2], 3]`は以下のような抽象構文木を表している．

 <img src="img/ast.png" width=50%>

 ## 式の値を計算する
 実験1と同様にexpr0を改造して式の値を電卓式に左から計算するプログラムをつくるのは簡単である（前回の`csv_sum`を参考にすればよい）．
 ここでは別の方法として抽象構文木をもとに式の値を計算する．抽象構文木を葉から根に向けてたどっていけば根に到達したときには式の値が得られる．以下はこのアイデアをプログラムにしたものである．

 （補足) `//`は切り捨てでわり算の結果を整数で返すPythonの演算子である．

In [0]:
def calc(ast):
    # 葉（整数）の場合そのまま返す
    if isinstance(ast, int):  
        return ast
    # 葉でないときは先に求めておいた左右の部分木の値をたしたりひいたりしながら根に向けて上昇する．
    [op, ast_left, ast_right] = ast 
    if op == '+':
        return calc(ast_left) + calc(ast_right)
    if op == '-':
        return calc(ast_left) - calc(ast_right)
    if op == '*':
        return calc(ast_left) * calc(ast_right)
    if op == '/':
        return calc(ast_left) // calc(ast_right)        

In [0]:
calc([
    '*',
    [
        '+',        
        1,
        2
    ],
    3
])

 この`calc`を`expr0`とつなげて流れ作業させれば式の値が求められる．

In [0]:
@parser_do
def calc_expr0(run):
    return calc(run(expr0))

In [0]:
calc_expr0("1+2*3") 

In [0]:
calc_expr0("1+3*4-5/2")

 ## 演算子の結合順位を考慮した改良
 普通の数学の習慣にしたがい演算子$*,/$を$+,-$より先に計算できるように数式を式(expr)と項(term)の2種類に分けて考える（いわゆる「多項式」の考え方）．
 $$
 \begin{align}
 \mbox{項} &\to \mbox{数} \ ((* \mid /) \ \mbox{数})^* \\
 \mbox{式} &\to \mbox{項} \ ((+ \mid -) \ \mbox{項})^* \\
 \end{align}
 $$

 上記の`expr0`は以下のように`term`と`expr`に分離される．

In [0]:
@parser_do
def term(run):
    a = run(num)
    while True:
        r = run(pat("[*/]") & num, nullable=True) 
        if r is None: break
        [op, b] = r
        a = [op, a, b]
    return a

@parser_do
def expr(run):
    a = run(term)
    while True:
        r = run(pat("[+-]") & term, nullable=True) 
        if r is None: break
        [op, b] = r
        a = [op, a, b]
    return a

 今度は数学の習慣に正しくしたがってかけ算がたし算よりも先にまとめられていることが分かる．

In [0]:
expr("1+2*3")

 同じ優先順位の演算が連続するときは左から順にまとめられる．

In [0]:
expr("1-2-3-4")

 `calc`をそのまま再利用して`expr`とつなげて流れ作業させれば式の値が求められる．

In [0]:
@parser_do
def calc_expr(run):
    return calc(run(expr))

In [0]:
calc_expr("1+2*3") 

In [0]:
calc_expr("1+3*4-5/2")

 ## かっこで囲った式の導入
 式の一部をかっこで囲って計算の順序をコントロールできる（例：$(1 + 2) * 3 = 9 \not= 7$．
 かっこでかこった式は単体の数と同じく「これ以上分割できない式」（因子(factor)）とみなされる．
 項(term)は因子を`*`, `/`で区切ってならべたものになる．式(expr)の定義は変更なし．
 $$
 \begin{align}
 \mbox{因子} &\to \mbox{数} \\
             &\ \ \mid\ \ \mbox{'('} \ \mbox{式}\ \mbox{')'} \\
 \mbox{項} &\to \mbox{因子} \ ((* \mid /) \ \mbox{因子})^* \\
 \mbox{式} &\to \mbox{項} \ ((+ \mid -) \ \mbox{項})^* \\
 \end{align}
 $$

In [0]:
@parser_do
def factor(run):
    return run(num | word("(") >> expr << word(")"))

@parser_do
def term(run):
    a = run(factor)
    while True:
        r = run(pat("[*/]") & factor, nullable=True) 
        if r is None: break
        [op, b] = r
        a = [op, a, b]
    return a

 かっこで囲った部分がひとまとめに扱われていることがわかる．

In [0]:
expr("1 + 2 * 3")

In [0]:
expr("(1 + 2) * 3")

 やはり`calc`は再利用できる．上記の`calc_expr`をそのまま使って以下の例を実行すると，かっこの部分が先に計算されていることがわかる．

In [0]:
calc_expr("1 + 2 * 3")

In [0]:
calc_expr("(1 + 2) * 3")

 ## べき乗演算の導入
 数学の習慣にしたがえばべき乗は`*`や`/`よりも優先して結合し先に計算される．例えば，$2*2^2 = 2*4 \not= 4^2$，$2^2*2 = 4*2 \not= 2^4$である．
 また他の演算子とは異なり右結合である．例えば，$2^{3^2} = 2^9 \not= 8^2$である．
 以下，$x^n$を`x^n`と書く．以上のことを文法でまとめると以下のようになる．
 $$
 \begin{align}
 \mbox{べき乗式} &\to \mbox{因子} \ \mbox{'^'}\ \mbox{べき乗式} \\
                 &\ \ \mid\ \ \mbox{因子} \\
 \mbox{項} &\to \mbox{べき乗式} \ ((* \mid /) \ \mbox{べき乗式})^* \\
 \end{align}
 $$
 因子(factor)と式(expr)の定義は以前と同じ．

 ### [課題2]
 べき乗を含む式の計算ができるように必要な変更・追加をおこなえ．以下は指針：
 1. べき乗式を切りだすパーサ`power_expr`をつくる
 2. 項を切り出すパーサ`term`を少し修正し`factor`の代わりに`power_expr`を用いるように変更
 3. `calc`にべき乗の計算を追加（Pythonでは`x^n`を`x ** n`と書くので注意）．

 [1のヒント] $\mbox{因子} \ \mbox{'^'}\ \mbox{べき乗式}$の切り出しに失敗したときだけバックアップ（下支え）で$\mbox{因子}$を切り出すようにするとよい． あるいは（同じことだが），$\mbox{因子} \ \mbox{'^'}\ \mbox{べき乗式}$を切り出すパーサを作っておいて`|`で`factor`と並列接続してもよい．

In [0]:
calc_expr("2^3^2")  # == 512

In [0]:
calc_expr("2*2^2")  # == 8       

In [0]:
calc_expr("2^2*2")  # == 8      

In [0]:
calc_expr("2^2+2")  # == 6

In [0]:
calc_expr("2^(2+2)")  # == 16

In [0]:
calc_expr("2+2^2")  # == 6

In [0]:
calc_expr("(2+2)^2")  # == 16                        